# **Data Exploration Notebook**

---

# Change working directory

Accessing the current directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Making sure working in the child of the workspace directory

In [ ]:
os.chdir('/workspaces/milestone-project-heritage-housing-issues')
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Explore the Kaggle Data

* Load Kaggle Data

In [ ]:
import pandas as pd
allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']
df = pd.read_csv(
    f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/"
    f"house_prices_records.csv", na_values=allowed_nans, keep_default_na=False)
df.head()

* Run Profile Report

In [ ]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

---

## Correlation and PPS Analysis

* Change object type data to numerical data

In [ ]:
df['BsmtExposure'] = df['BsmtExposure'].replace(
    {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4})
df['BsmtFinType1'] = df['BsmtFinType1'].replace(
    {'None': 0, 'Unf': 1, 'LwQ': 2, 'BLQ': 3, 'Rec': 4, 'ALQ': 5, 'GLQ': 6})
df['GarageFinish'] = df['GarageFinish'].replace(
    {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3})
df['KitchenQual'] = df['KitchenQual'].replace(
    {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4})

* Custom function from the Code Institute Wakthrough Project 02

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='plasma',
                    annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r',
                         annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(
        columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query(
        "ppscore < 1").filter(['ppscore']).describe().T
    print(
        "PPS threshold - check PPS score IQR to decide threshold for heatmap\n"
         )
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman,
                      pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are "
          "correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are "
          "correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold,
                 figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous "
          "variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold,
                 figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two "
          f"columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect "
          f"predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize,
                font_annot=font_annot)

Calculate Correlations and Power Predictive Score

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

Heatmaps

In [ ]:
DisplayCorrAndPPS(df_corr_pearson=df_corr_pearson,
                  df_corr_spearman=df_corr_spearman,
                  pps_matrix=pps_matrix,
                  CorrThreshold=0.4, PPS_Threshold=0.2,
                  figsize=(12, 10), font_annot=10)

The PPS heatmap shows that none of the variables have strong predictive power for sale price therefore, I will concentrate on spearman and pearson correlation

* Top five correlated attributes, to SalePrice

In [ ]:
df_spearman = df.corr(method='spearman')['SalePrice'].sort_values(
    key=abs, ascending=False)[1:].head()
df_pearson = df.corr(method='pearson')['SalePrice'].sort_values(
    key=abs, ascending=False)[1:].head()
set(df_pearson[:4].index.to_list() + df_spearman[:4].index.to_list())

Therefore I will investigate how the following attributes affect SalePrice:

* GarageArea
* GrLivArea
* KitchenQual
* OverallQual
* YearBuilt

In [ ]:
vars_to_study = [
    'GarageArea', 'GrLivArea', 'KitchenQual',
    'OverallQual', 'YearBuilt'
    ]
vars_to_study

In [ ]:
for col in vars_to_study:
    ax = sns.regplot(
        data=df, x=col, y="SalePrice", scatter_kws={
            "color": "purple"}, line_kws={"color": "red"})
    plt.ylabel('SalePrice')
    plt.xlabel(col)
    plt.title(f"{col}", fontsize=20, y=1.1)
    plt.show()

---

# Conclusions

* Overall quality has the higest impact on SalePrice
* Kitchen quality also impacts SalePrice with higher quality getting a higher price
* Newer homes have higher SalePrice
* Larger garage and living areas have higher SalePrice - a larger living area has the most impact 